<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/Bradley_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext
from torchtext.data import Field
import spacy

from spacy.symbols import ORTH
my_tok = spacy.load('en')

def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

QUERY = Field(lower=True, tokenize=spacy_tok)
RESPONSE = Field(lower=True, tokenize=spacy_tok, is_target=True, init_token='<bos>', eos_token='<eos>')

In [0]:
import torch
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
my_tok.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])

In [0]:
from torchtext.data import TabularDataset

train_dataset = TabularDataset(path="./simpleR21.csv", format="CSV",
                               fields=[("query", QUERY),("response", RESPONSE)],
                               csv_reader_params={"delimiter":'\t'})

In [7]:
QUERY.build_vocab(train_dataset)
RESPONSE.build_vocab(train_dataset)
print(QUERY.vocab.stoi['film'])
print(QUERY.vocab.itos[33])
print(len(QUERY.vocab))

13
religion
66


In [0]:
from torchtext.data import BucketIterator

train_iterator = BucketIterator(dataset= train_dataset, batch_size=16
                                ,device=device
                                ,sort_key=lambda x: data.interleave_keys(len(x.query), len(x.response))
                                , repeat = False)

In [9]:
for batch in (iter(train_iterator)):
  print(batch.query.shape)

torch.Size([7, 16])
torch.Size([6, 16])
torch.Size([12, 16])
torch.Size([7, 16])


In [0]:
import torch.nn as nn
class Bradley(nn.Module):
  def __init__(self, src_voc_sze, trg_voc_sze, hid_sze, num_head, num_enc, num_dec):
    super(Bradley, self).__init__()
    self.hid_sze = hid_sze
    self.src_word_embedding = nn.Embedding(src_voc_sze, self.hid_sze)
    self.trg_word_embedding = nn.Embedding(trg_voc_sze, self.hid_sze)
    self.num_head = num_head
    self.transformer = nn.Transformer(self.hid_sze, self.num_head, num_enc, num_dec)
    self.fc = nn.Linear(self.hid_sze, trg_voc_sze)
  
  def forward(self, src, trg):
    temp_src = self.src_word_embedding(src)
    temp_trg = self.src_word_embedding(trg)
    temp = self.transformer(temp_src, temp_trg)
    return self.fc(temp)


In [0]:
bradley_model = Bradley(src_voc_sze=len(QUERY.vocab), trg_voc_sze=len(RESPONSE.vocab),
                        hid_sze=256, num_head=4,
                        num_enc=4, num_dec=2)
bradley_model = bradley_model.to(device)

In [75]:
criterion = nn.CrossEntropyLoss()
lr = 0.5
optimizer = torch.optim.SGD(bradley_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
epoch_number = 10

for epoch in range(1, epoch_number+1):
  bradley_model.train()
  for batch in iter(train_iterator):
    src = batch.query
    trg = batch.query
    out = bradley_model(src, trg)
    loss = criterion(out.view(-1, out.shape[2]), trg.view(-1))
    print(loss.item())
    loss.backward()
    torch.nn.utils.clip_grad_norm_(bradley_model.parameters(), 0.5)
    optimizer.step()

4.96686315536499
3.5264992713928223
2.080118179321289
3.0013229846954346
1.6152182817459106
2.7996318340301514
1.9602636098861694
1.4487637281417847
1.5394340753555298
1.2498375177383423
0.8729048371315002
1.1534901857376099
0.9681708216667175
0.6927459836006165
0.7604721784591675
0.6124193668365479
0.5347681641578674
0.6757655143737793
0.46241116523742676
0.5170096755027771
0.3575325012207031
0.3833262026309967
0.3883305490016937
0.2246009260416031
0.22116543352603912
0.14700908958911896
0.2167665958404541
0.21887743473052979
0.06838489323854446
0.13479556143283844
0.17004738748073578
0.07649350166320801
0.04738254472613335
0.06227310746908188
0.07413377612829208
0.031506408005952835
0.01874273084104061
0.016057291999459267
0.02504136599600315
0.038459498435258865
